# Classifying Text and Analysing Tweets

1. Get tweets using Twitter API
2. Load Model
3. Classify Tweets
---

## 01 Get Tweets using Twitter API

In [1]:
import requests
import os
import json

In [2]:
# bearer_token to be saved as ENV variable on deployment

class TwitterAPI:
    def __init__(self):
        self.bearer_token = "<YOUR_BEARER_TOKEN>"
        self.headers = {"Authorization": "Bearer {}".format(self.bearer_token)}
    
    def UserInfoUrl(self, uname):
        usernames = f"usernames={uname}"
        user_fields = "user.fields=description,created_at,profile_image_url"
        return f"https://api.twitter.com/2/users/by?{usernames}&{user_fields}"
  
    def getUserInfo(self, uname):
        url = self.UserInfoUrl(uname)
        response = requests.request("GET", url, headers=self.headers)
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(
                "Request returned an error: {} {}".format(
                    response.status_code, response.text
                )
            )
        res = response.json()
        return res['data'][0]
    
    def getTweets(self, uid):
        params = {"tweet.fields": "created_at", "max_results":"100"}
        url = f"https://api.twitter.com/2/users/{uid}/tweets"
        response = requests.request("GET", url, headers=self.headers, params=params)
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(
                "Request returned an error: {} {}".format(
                    response.status_code, response.text
                )
            )
        return response.json()['data']

In [3]:
tw = TwitterAPI()

In [4]:
userInfo = tw.getUserInfo("nytimes") # user Info Newyork Times
tweets = tw.getTweets(userInfo["id"]) # Tweets

200
200


In [5]:
userInfo

{'name': 'The New York Times',
 'username': 'nytimes',
 'profile_image_url': 'https://pbs.twimg.com/profile_images/1098244578472280064/gjkVMelR_normal.png',
 'description': 'News tips? Share them here: https://t.co/ghL9OoYKMM',
 'created_at': '2007-03-02T20:41:42.000Z',
 'id': '807095'}

In [6]:
tweets[0:5]

[{'created_at': '2021-06-26T06:00:10.000Z',
  'id': '1408666376492011520',
  'text': 'Black creators on TikTok have begun a viral campaign — echoing messages in the music video for Megan Thee Stallion’s latest single — that aims to draw attention to how essential they are to the platform. “Black people carry the app,” one creator said. https://t.co/DKBs9xP9sA'},
 {'created_at': '2021-06-26T05:30:06.000Z',
  'id': '1408658810605035522',
  'text': 'RT @nytimesworld: President Iván Duque of Colombia said he was inside a helicopter that was shot at Friday afternoon, in an attack that he…'},
 {'created_at': '2021-06-26T05:00:14.000Z',
  'id': '1408651292025802756',
  'text': 'Swipe through to watch the best moments from late night this week. 📺 →\nhttps://t.co/3vyD5KSaq2'},
 {'created_at': '2021-06-26T04:30:08.000Z',
  'id': '1408643719763476480',
  'text': "RT @christophclarey: 21 American women in the Wimbledon singles draw and that's without the injured Jennifer Brady.\n\n21 is the most a

## 02 Load Model

In [7]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.models import load_model

import io
import json

from termcolor import colored

In [8]:
# Model Paramaters (Should be the same when training the model)
vocab_size = 8000
embedding_dim = 32
max_length = 40
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [9]:
# Load Model
model = tf.keras.models.load_model('./SentimentModel.h5')

In [10]:
# Load Tokenizer
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [11]:
def predictSentiment(x):
    sequences = tokenizer.texts_to_sequences(x)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    return model.predict(padded)

In [12]:
# Testing the loaded model
predictSentiment(["I love python", "I hate You"])

array([[0.95154464],
       [0.11488223]], dtype=float32)

## 03 Classify Tweets

In [13]:
len(tweets)

100

In [14]:
# for this test we will only use 10 tweets
tweets = tweets[0:10]

In [15]:
# We only need test
tweets = [i["text"] for i in tweets]

In [16]:
tweets

['Black creators on TikTok have begun a viral campaign — echoing messages in the music video for Megan Thee Stallion’s latest single — that aims to draw attention to how essential they are to the platform. “Black people carry the app,” one creator said. https://t.co/DKBs9xP9sA',
 'RT @nytimesworld: President Iván Duque of Colombia said he was inside a helicopter that was shot at Friday afternoon, in an attack that he…',
 'Swipe through to watch the best moments from late night this week. 📺 →\nhttps://t.co/3vyD5KSaq2',
 "RT @christophclarey: 21 American women in the Wimbledon singles draw and that's without the injured Jennifer Brady.\n\n21 is the most at Wimb…",
 'A New York Times photographer witnessed Afghanistan’s elite forces on a recent night raid as they disrupted Taliban operations in a volatile province. https://t.co/WExdcVx88M',
 '8 new books recommended by critics and editors at The New York Times. https://t.co/hqDUmkZkPj',
 'The chief said that the bystander’s actions saved 

In [17]:
sentiment = predictSentiment(tweets)

In [18]:
sentiment

array([[0.9997283 ],
       [0.6708385 ],
       [0.9316113 ],
       [0.00956056],
       [0.9948553 ],
       [0.99004555],
       [0.18096861],
       [0.08231688],
       [0.00926277],
       [0.93989396]], dtype=float32)

In [19]:
for tweet,sent in zip(tweets, sentiment):
    if sent > 0.6:
        print(colored(tweet, 'green'))
    elif sent < 0.4:
        print(colored(tweet, 'red'))
    else :
        print(colored(tweet, 'blue'))

# RED => NEGATIVE
# GREEN => POSITIVE

Black creators on TikTok have begun a viral campaign — echoing messages in the music video for Megan Thee Stallion’s latest single — that aims to draw attention to how essential they are to the platform. “Black people carry the app,” one creator said. https://t.co/DKBs9xP9sA
RT @nytimesworld: President Iván Duque of Colombia said he was inside a helicopter that was shot at Friday afternoon, in an attack that he…
Swipe through to watch the best moments from late night this week. 📺 →
https://t.co/3vyD5KSaq2
RT @christophclarey: 21 American women in the Wimbledon singles draw and that's without the injured Jennifer Brady.

21 is the most at Wimb…
A New York Times photographer witnessed Afghanistan’s elite forces on a recent night raid as they disrupted Taliban operations in a volatile province. https://t.co/WExdcVx88M
8 new books recommended by critics and editors at The New York Times. https://t.co/hqDUmkZkPj
The chief said that the bystander’s actions saved lives and that what he did “c

In [25]:
for i in sentiment:
    print(i[0])

0.9997283
0.6708385
0.9316113
0.009560555
0.9948553
0.99004555
0.18096861
0.082316875
0.00926277
0.93989396


In [41]:
out = []
for i,j in zip(tweets, sentiment):
    out.append( {"text":i, "sentiment":str(j[0])} )

In [46]:
json.dumps(out)

'[{"text": "Black creators on TikTok have begun a viral campaign \\u2014 echoing messages in the music video for Megan Thee Stallion\\u2019s latest single \\u2014 that aims to draw attention to how essential they are to the platform. \\u201cBlack people carry the app,\\u201d one creator said. https://t.co/DKBs9xP9sA", "sentiment": "0.9997283"}, {"text": "RT @nytimesworld: President Iv\\u00e1n Duque of Colombia said he was inside a helicopter that was shot at Friday afternoon, in an attack that he\\u2026", "sentiment": "0.6708385"}, {"text": "Swipe through to watch the best moments from late night this week. \\ud83d\\udcfa \\u2192\\nhttps://t.co/3vyD5KSaq2", "sentiment": "0.9316113"}, {"text": "RT @christophclarey: 21 American women in the Wimbledon singles draw and that\'s without the injured Jennifer Brady.\\n\\n21 is the most at Wimb\\u2026", "sentiment": "0.009560555"}, {"text": "A New York Times photographer witnessed Afghanistan\\u2019s elite forces on a recent night raid as they 